## Computer Vision using VGG16 CNN architecture model

#### Matthew Yeseta, Master Data Science

In [2]:
!pip install opencv-python

  Obtaining dependency information for opencv-python from https://files.pythonhosted.org/packages/c7/ec/9dabb6a9abfdebb3c45b0cc52dec901caafef2b2c7e7d6a839ed86d81e91/opencv_python-4.9.0.80-cp37-abi3-win_amd64.whl.metadata
   ---------------------------------------- 0.0/38.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.6 MB 960.0 kB/s eta 0:00:41
   ---------------------------------------- 0.1/38.6 MB 1.4 MB/s eta 0:00:28
   ---------------------------------------- 0.3/38.6 MB 2.0 MB/s eta 0:00:20
   ---------------------------------------- 0.4/38.6 MB 2.4 MB/s eta 0:00:17
    --------------------------------------- 0.6/38.6 MB 2.6 MB/s eta 0:00:15
    --------------------------------------- 0.7/38.6 MB 2.6 MB/s eta 0:00:15
    --------------------------------------- 0.9/38.6 MB 2.8 MB/s eta 0:00:14
    --------------------------------------- 1.0/38.6 MB 2.6 MB/s eta 0:00:15
   - -------------------------------------- 1.0/38.6 MB 2.5 MB/s eta 0:00:16
   - ---------

In [ ]:
from IPython.display import clear_output
!pip install imutils
clear_output()

In [ ]:
import numpy as np 
import warnings
from tqdm import tqdm
import sys
import cv2
import os
from os import listdir
from os.path import isfile, join
import shutil
import itertools
from itertools import chain
import imutils
import matplotlib.pyplot as plt
import numpy as np 
from numpy import expand_dims
import pandas as pd 
import math
import seaborn as sns
import umap
from PIL import Image
from scipy import misc
from scipy import misc
from random import shuffle
from collections import Counter
from matplotlib import pyplot 


from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, f1_score, cohen_kappa_score, roc_auc_score
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.datasets import make_circles

import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from plotly import tools

import keras
from keras import backend as K
from keras import layers
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, MaxPooling2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.layers import Input, Activation, Dropout, Flatten, Dense
from keras.models import Model, Sequential, load_model
from keras.optimizers import Adam, RMSprop
from keras.callbacks import EarlyStopping, ModelCheckpoint

from keras.applications.vgg16 import VGG16, preprocess_input
from keras.applications.vgg16 import preprocess_input

from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import Model
from keras.utils.np_utils import to_categorical

from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

import tensorflow as tf

init_notebook_mode(connected=True)
RANDOM_SEED = 123

In [ ]:
!apt-get install tree
!mkdir TRAIN TEST VAL TRAIN/YES TRAIN/NO TEST/YES TEST/NO VAL/YES VAL/NO
!tree -d

In [ ]:
IMG_PATH = '../input/brain-mri-images-for-brain-tumor-detection/brain_tumor_dataset/'

for CLASS in os.listdir(IMG_PATH):
    if not CLASS.startswith('.'):
        IMG_NUM = len(os.listdir(IMG_PATH + CLASS))
        for (n, FILE_NAME) in enumerate(os.listdir(IMG_PATH + CLASS)):
            img = IMG_PATH + CLASS + '/' + FILE_NAME
            if n < 0.8*IMG_NUM:
                shutil.copy(img, 'TRAIN/'+ CLASS.upper() + '/' + FILE_NAME)
            elif n < 5:
                shutil.copy(img, 'TEST/' + CLASS.upper() + '/' + FILE_NAME)
            else:
                shutil.copy(img, 'VAL/'+ CLASS.upper() + '/' + FILE_NAME)

In [ ]:
def load_data(dir_path, img_size=(100,100)):
    x_data = []
    y_data = []
    i = 0
    labels = dict()
    for path in tqdm(sorted(os.listdir(dir_path))):
        if not path.startswith('.'):
            labels[i] = path
            for file in os.listdir(dir_path + path):
                if not file.startswith('.'):
                    img = cv2.imread(dir_path + path + '/' + file)
                    X.append(img)
                    y.append(i)
            i += 1
    X = np.array(x_data)
    y = np.array(y_data)
    print(f'{len(X)} images loaded from {dir_path} directory.')
    return X, y, labels



def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):

    plt.figure(figsize = (6,6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)
    if normalize:
        cmatrix = cm.astype('float') / cmatrix.sum(axis=1)[:, np.newaxis]

    cmatrix = np.round(cmatrix,2)
    thresh = cmatrix.max() / 2.
    for i, j in itertools.product(range(cmatrix.shape[0]), range(cmatrix.shape[1])):
        plt.text(j, i, cmatrix[i, j],
                 horizontalalignment="center",
                 color="white" if cmatrix[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

In [ ]:
def crop_imgs(set_name, add_pixels_value=0):

    set_new = []
    for img in set_name:
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY); gray = cv2.GaussianBlur(gray, (5, 5), 0)

        img_thresh = cv2.threshold(gray, 
                               45, 255, 
                               cv2.THRESH_BINARY)[1]
        cv_thresh = cv2.erode(img_thresh, None, iterations=2)
        cv_thresh = cv2.dilate(img_thresh, None, iterations=2)
        cnts = cv2.findContours(cv_thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts = imutils.grab_contours(cnts)
        cnts_C = max(cnts, key=cv2.contourArea)
        extLeft = tuple(cnts_C[cnts_C[:, :, 0].argmin()][0]); extRight = tuple(cnts_C[cnts_C[:, :, 0].argmax()][0])
        extTop = tuple(cnts_C[cnts_C[:, :, 1].argmin()][0]); extBot = tuple(cnts_C[cnts_C[:, :, 1].argmax()][0])

        ADD_PIXELS = add_pixels_value
        new_img = img[extTop[1]-ADD_PIXELS:extBot[1]+ADD_PIXELS, extLeft[0]-ADD_PIXELS:extRight[0]+ADD_PIXELS].copy()
        set_new.append(new_img)

    return np.array(set_new)

In [ ]:
img = cv2.imread('../input/brain-mri-images-for-brain-tumor-detection/brain_tumor_dataset/yes/Y108.jpg')
img = cv2.resize(
            img,
            dsize=(224,224),
            interpolation=cv2.INTER_CUBIC
        )
gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
gray = cv2.GaussianBlur(gray, (5, 5), 0)

img_thresh = cv2.threshold(gray, 
                       45, 255, 
                       cv2.THRESH_BINARY)[1]
cv_thresh = cv2.erode(img_thresh, None, iterations=2)
cv_thresh = cv2.dilate(img_thresh, None, iterations=2)

cnts = cv2.findContours(cv_thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
cnts_C = max(cnts, key=cv2.contourArea)
extLeft = tuple(cnts_C[cnts_C[:, :, 0].argmin()][0]); extRight = tuple(cnts_C[cnts_C[:, :, 0].argmax()][0])
extTop = tuple(cnts_C[cnts_C[:, :, 1].argmin()][0]); extBot = tuple(cnts_C[cnts_C[:, :, 1].argmax()][0])
img_cnt = cv2.drawContours(img.copy(), [cnts_C], -1, (0, 255, 255), 4)
img_pnt = cv2.circle(img_cnt.copy(), extLeft, 8, (0, 0, 255), -1)
img_pnt = cv2.circle(img_pnt, extRight, 8, (0, 255, 0), -1)
img_pnt = cv2.circle(img_pnt, extTop, 8, (255, 0, 0), -1)
img_pnt = cv2.circle(img_pnt, extBot, 8, (255, 255, 0), -1)


ADD_PIXELS = 0
new_img = img[extTop[1]-ADD_PIXELS:extBot[1]+ADD_PIXELS, extLeft[0]-ADD_PIXELS:extRight[0]+ADD_PIXELS].copy()

In [ ]:
def preprocess_imgs(set_name, img_size):

    set_new = []
    for img in set_name:
        img = cv2.resize(
            img,
            dsize=img_size,
            interpolation=cv2.INTER_CUBIC
        )
        set_new.append(preprocess_input(img))
    return np.array(set_new)

In [ ]:
TRAIN_DIR = 'TRAIN/'
TEST_DIR = 'TEST/'
VAL_DIR = 'VAL/'

X_train, y_train, labels = load_data(TRAIN_DIR, (224,224))
X_test, y_test, _ = load_data(TEST_DIR, (224,224))
X_val, y_val, _ = load_data(VAL_DIR, (224,224))

X_train_crop = crop_imgs(set_name=X_train)
X_val_crop = crop_imgs(set_name=X_val)
X_test_crop = crop_imgs(set_name=X_test)

X_train_prep = preprocess_imgs(set_name=X_train_crop, img_size=(224,224))
X_test_prep = preprocess_imgs(set_name=X_test_crop, img_size=(224,224))
X_val_prep = preprocess_imgs(set_name=X_val_crop, img_size=(224,224))

!mkdir TRAIN_CROP TEST_CROP VAL_CROP TRAIN_CROP/YES TRAIN_CROP/NO TEST_CROP/YES TEST_CROP/NO VAL_CROP/YES VAL_CROP/NO


In [ ]:
def save_new_images(x_set, y_set, folder_name):
    i = 0
    for (img, imclass) in zip(x_set, y_set):
        if imclass == 0:
            cv2.imwrite(folder_name+'NO/'+str(i)+'.jpg', img)
        else:
            cv2.imwrite(folder_name+'YES/'+str(i)+'.jpg', img)
        i += 1

In [ ]:
save_new_images(X_train_crop, y_train, folder_name='TRAIN_CROP/')
save_new_images(X_val_crop, y_val, folder_name='VAL_CROP/')
save_new_images(X_test_crop, y_test, folder_name='TEST_CROP/')

In [ ]:

generator_datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.05,
    height_shift_range=0.05,
    rescale=1./255,
    shear_range=0.05,
    brightness_range=[0.1, 1.5],
    horizontal_flip=True,
    vertical_flip=True
)

In [ ]:

x = X_train_crop[0]  
x = x.reshape((1,) + x.shape) 

i = 0
for batch in generator_datagen.flow(x, batch_size=1, save_to_dir='preview', save_prefix='aug_img', save_format='jpg'):
    i += 1
    if i > 20:
        break 

In [ ]:
plt.imshow(X_train_crop[0])
plt.xticks([])
plt.yticks([])
plt.title('Original Image')
plt.show()

plt.figure(figsize=(15,6))
i = 1
for img in os.listdir('preview/'):
    img = cv2.cv2.imread('preview/' + img)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.subplot(3,7,i)
    plt.imshow(img)
    plt.xticks([])
    plt.yticks([])
    i += 1
    if i > 3*7:
        break
plt.suptitle('Augemented Images')
plt.show()

In [ ]:
TRAIN_DIR = 'TRAIN_CROP/'
VAL_DIR = 'VAL_CROP/'

train_datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    brightness_range=[0.5, 1.5],
    horizontal_flip=True,
    vertical_flip=True,
    preprocessing_function=preprocess_input
)

test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    color_mode='rgb',
    target_size=(224,224),
    batch_size=32,
    class_mode='binary',
    seed=RANDOM_SEED
)

validation_generator = test_datagen.flow_from_directory(
    VAL_DIR,
    color_mode='rgb',
    target_size=(224,224),
    batch_size=16,
    class_mode='binary',
    seed=RANDOM_SEED
)

In [ ]:

vgg16_weight_path = '../input/keras-pretrained-models/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'
vgg = VGG16(
    weights=vgg16_weight_path,
    include_top=False, 
    input_shape=(224,224) + (3,)
)

In [ ]:
f = plt.figure(figsize=(16,16))

model = VGG16()

model = Model(inputs=model.inputs, outputs=model.layers[1].output)
model.summary()
img = img_to_array(X_val_prep[43])
img = expand_dims(img, axis=0)
img = preprocess_input(img)
feature_maps = model.predict(img)
square = 8
ix = 1

for _ in range(square):
    for _ in range(square):
        ax = pyplot.subplot(square, square, ix)
        ax.set_xticks([])
        ax.set_yticks([])
        pyplot.imshow(feature_maps[0, :, :, ix-1], cmap='viridis')
        ix += 1

pyplot.show()


In [ ]:

NUM_CLASSES = 1

vgg16 = Sequential()
vgg16.add(vgg)
vgg16.add(layers.Dropout(0.3))
vgg16.add(layers.Flatten())
vgg16.add(layers.Dropout(0.5))
vgg16.add(layers.Dense(NUM_CLASSES, activation='sigmoid'))

vgg16.layers[0].trainable = False

vgg16.compile(
    loss='binary_crossentropy',
    optimizer=RMSprop(lr=1e-4),
    metrics=['accuracy']
)
vgg16.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(lr=0.0003, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False), metrics=["accuracy"])

vgg16.summary()

In [ ]:
plt.rcParams.update({'figure.max_open_warning': 0})
model = VGG16()

ixs = [2, 5, 9, 13, 17]
outputs = [model.layers[i].output for i in ixs]
model = Model(inputs=model.inputs, outputs=outputs)

img = img_to_array(X_val_prep[43])
img = expand_dims(img, axis=0)
img = preprocess_input(img)
feature_maps = model.predict(img)

square = 8
for fmap in feature_maps:
    ix = 1
    for _ in range(square):
        plt.figure(figsize=(64,64))
        for _ in range(square):
            ax = pyplot.subplot(square, square, ix)
            ax.set_xticks([])
            ax.set_yticks([])
            plt.imshow(fmap[0, :, :, ix-1], cmap='viridis')
            ix += 1
        
plt.show()

In [ ]:
import time

start = time.time()

vgg16_history = vgg16.fit_generator(
    train_generator,
    steps_per_epoch=50,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=30,
)

end = time.time()
print(end - start)


In [ ]:

predictions = vgg16.predict(X_test_prep)
predictions = [1 if x>0.5 else 0 for x in predictions]

_, train_acc = vgg16.evaluate(X_val_prep, y_val, verbose=0)
_, test_acc = vgg16.evaluate(X_test_prep, y_test, verbose=0)

In [ ]:
accuracy = accuracy_score(y_test, predictions)
print('Accuracy: %f' % accuracy)
precision = precision_score(y_test, predictions)
recall = recall_score(y_test, predictions)
print('Recall: %f' % recall)
f1 = f1_score(y_test, predictions)
print('F1 score: %f' % f1)

kappa = cohen_kappa_score(y_test, predictions)
print('Cohens kappa: %f' % kappa)

auc = roc_auc_score(y_test, predictions)
print('ROC AUC: %f' % auc)

matrix = confusion_matrix(y_test, predictions)
print(matrix)


In [ ]:
pyplot.figure(figsize=(12,12))

pyplot.subplot(211)
pyplot.title('Vgg16 Loss')
pyplot.plot(vgg16_history.history['loss'], label='train')
pyplot.plot(vgg16_history.history['val_loss'], label='Validation')
pyplot.legend()

pyplot.subplot(212)
pyplot.title('Vgg16 Accuracy')
pyplot.plot(vgg16_history.history['acc'], label='train')
pyplot.plot(vgg16_history.history['val_acc'], label='Validation')
pyplot.legend()
pyplot.show()


In [ ]:
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))

In [ ]:
def ModelGraphTrainngSummary(history,N,model_name):

    sys.stdout.flush()
    matplotlib.use("Agg")
    matplotlib.pyplot.style.use("ggplot")
    matplotlib.pyplot.figure()
    matplotlib.pyplot.plot(np.arange(0, N), history.history["loss"], label="train_loss")
    matplotlib.pyplot.plot(np.arange(0, N), history.history["val_loss"], label="val_loss")

    matplotlib.pyplot.title("Training Loss and Accuracy on Brain Tumor Classification")
    matplotlib.pyplot.xlabel("Epoch #")
    matplotlib.pyplot.ylabel("Loss/Accuracy of "+model_name)
    matplotlib.pyplot.legend(loc="lower left")
    matplotlib.pyplot.savefig("plot.png")

In [ ]:
def ModelGraphTrainngSummaryAcc(history,N,model_name):

    print("Generating plots...")
    sys.stdout.flush()
    matplotlib.use("Agg")
    matplotlib.pyplot.style.use("ggplot")
    matplotlib.pyplot.figure()
    matplotlib.pyplot.plot(np.arange(0, N), history.history["acc"], label="train_acc")
    matplotlib.pyplot.plot(np.arange(0, N), history.history["val_acc"], label="val_acc")
    matplotlib.pyplot.title("Training Loss and Accuracy on Brain Tumor Classification")
    matplotlib.pyplot.xlabel("Epoch #")
    matplotlib.pyplot.ylabel("Accuracy of "+ model_name)
    matplotlib.pyplot.legend(loc="lower left")
    matplotlib.pyplot.savefig("plot.png")

In [ ]:
for x_model in [{'name':'VGG-16','history':history_1,'model':vgg16}]:
    ModelGraphTrainngSummary(x_model['history'],120,x_model['name'])
    ModelGraphTrainngSummaryAcc(x_model['history'],120,x_model['name'])
    
    predictions = x_model['model'].predict(X_val_prep)
    predictions = [1 if x>0.5 else 0 for x in predictions]

    accuracy = accuracy_score(y_val, predictions)
    print('Val Accuracy = %.2f' % accuracy)

    confusion_mtx = confusion_matrix(y_val, predictions) 
    cm = plot_confusion_matrix(confusion_mtx, classes = list(labels.items()), normalize=False)

In [ ]:
!rm -rf TRAIN TEST VAL TRAIN_CROP TEST_CROP VAL_CROP
vgg16.save('2023_VGG_model.h5')


